In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# the following line allows ipython to display plots
%matplotlib inline  

In [2]:
# read this csv file, remember to put the full path to 
# the directory where you saved the data
df = pd.read_csv('1429_1.csv')  # df is DataFrame object

/Users/ds070111/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Helper function that we will use to transform the rating to sentiment value
def label_sentiment (row, attribute):
    if row[attribute] >3 :
        return 1
    if row[attribute] <3 :
        return 3
    else:
        return 2

In [4]:
# Read data from CSV, filter the columns for the desired columns, 
df = pd.read_csv('1429_1.csv')  # df is DataFrame object
df['sentiment_label'] = df.apply (lambda row: label_sentiment(row, "reviews.rating"), axis=1)
df = df.filter(['reviews.text', 'sentiment_label'])
df = df.sample(frac=1)

In [5]:
df.head()

,reviews.text,sentiment_label
11795,gave as a gift along with the cover. person se...,1
23104,Had it for about year and would say it had goo...,1
4824,"Not the fastest for loading web pages, but app...",1
16157,I bought this as a gift for my smart 1 yr old ...,1
7485,I bought for my niece and she loves it. The fa...,1


In [6]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

import numpy as np 
import pandas as pd 
from time import time
import re
import string
import os
from pprint import pprint
import collections
import matplotlib.pyplot as plt
import seaborn as sns
# Import the classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import joblib

sns.set(style="darkgrid")
sns.set(font_scale=1.3)
pd.set_option('display.max_colwidth', -1)
nltk.download('stopwords')

<ipython-input-6-bffe6abf7db5>:32: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ds070111/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# Clean Text Class which will clea dataframe column which consists of text

class CleanText(BaseEstimator, TransformerMixin):
    # removing punctations
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space

        return str(input_text).translate(trantab)
    # Remove numbers
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    # Make it lowercase
    def to_lower(self, input_text):
        return input_text.lower()
    # Remove stop words
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    # Stemming
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    # Fit it
    def fit(self, X, y=None, **fit_params):
        return self
    # Transform the column
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

In [8]:
# Function that will create a bag of words give dataframe column
def create_bag_of_words(X):
    from sklearn.feature_extraction.text import CountVectorizer
    
    print ('Creating bag of words...')
    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    
    # In this example features may be single words or two consecutive words
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 ngram_range = (1,2), \
                                 max_features = 10000,
                                max_df = 0.5,
                                min_df =1) 

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings. The output is a sparse array
    train_data_features = vectorizer.fit_transform(X)
    
    # Convert to a NumPy array for easy of handling
    train_data_features = train_data_features.toarray()
    
    # tfidf transform
    from sklearn.feature_extraction.text import TfidfTransformer
    tfidf = TfidfTransformer()
    tfidf_features = tfidf.fit_transform(train_data_features).toarray()

    # Take a look at the words in the vocabulary
    vocab = vectorizer.get_feature_names()
    
    print ('Completed bag of words...')
    print(' Returning vectorizer, vocab, train_data_features, tfidf_features, tfidf.....')
    return vectorizer, vocab, train_data_features, tfidf_features, tfidf


In [9]:
from sklearn.linear_model import LogisticRegression
# train logistic regression
def train_LG(X, Y):
    print ("Training the logistic regression model...")
    ml_model = LogisticRegression(C = 0.2,penalty= 'l2', random_state = 0)
    ml_model.fit(X, Y)
    print ('Completed!')
    return ml_model

In [10]:
# train NBC classifier
from sklearn.naive_bayes import MultinomialNB
def train_NB(X, Y):
    print ("Training Naive Bayes model...")
    ml_model = MultinomialNB()
    ml_model.fit(X, Y)
    print ('Completed!')
    return ml_model

In [11]:
from sklearn import svm
def train_SVM(X, Y):
    print ("Training SVM...")
    ml_model = svm.SVC()
    ml_model.fit(X, Y)
    print ('Completed!')
    return ml_model

In [12]:
def accuracy(predicted_y, y_test): 
    correctly_identified_y = predicted_y == y_test
    accuracy = np.mean(correctly_identified_y) * 100
    print (accuracy)

In [13]:
def test_algorithm(algorithm):
    if(algorithm == "LG"):
        # Get the model
        ml_model = train_LG(tfidf_features, y_train)
        # vectorize the attributes of the test data after it has been cleaned
        test_data_features = vectorizer.transform(X_test['clean_text'])
        # TF
        test_data_tfidf_features = tfidf.fit_transform(test_data_features)
        #Predict 
        predicted_y = ml_model.predict(test_data_tfidf_features)
        #Accuracy
        accuracy(predicted_y, y_test)
        return ml_model
    elif(algorithm =="NB"):
        # Get the model
        ml_model = train_NB(tfidf_features, y_train)
        # vectorize the attributes of the test data after it has been cleaned
        test_data_features = vectorizer.transform(X_test['clean_text'])
        # TF
        test_data_tfidf_features = tfidf.fit_transform(test_data_features)
        #Predict 
        predicted_y = ml_model.predict(test_data_tfidf_features)
        #Accuracy
        accuracy(predicted_y, y_test)
        return ml_model
    else:
        # Get the model
        ml_model = train_SVM(tfidf_features, y_train)
        # vectorize the attributes of the test data after it has been cleaned
        test_data_features = vectorizer.transform(X_test['clean_text'])
        # TF
        test_data_tfidf_features = tfidf.fit_transform(test_data_features)
        #Predict 
        predicted_y = ml_model.predict(test_data_tfidf_features)
        #Accuracy
        accuracy(predicted_y, y_test)
        return ml_model

In [14]:
# Main Driver

# Clean the text column
ct = CleanText()
sr_clean = ct.fit_transform(df['reviews.text'])

# Create clean_text column
df_model = df
df_model['clean_text'] = sr_clean
df_model.columns.tolist()

# Split the data(cleaned, preprocessed)
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment_label', axis=1), df_model.sentiment_label, test_size=0.45, random_state=2)

vectorizer, vocab, train_data_features, tfidf_features, tfidf  = (
        create_bag_of_words(X_train['clean_text']))

vectorizer1 = vectorizer
tfidf1 = tfidf


Creating bag of words...
Completed bag of words...
 Returning vectorizer, vocab, train_data_features, tfidf_features, tfidf.....


In [15]:
print("Testing Logistic Regression....")
lg_model_1 = test_algorithm("LG")
print("Testing Naive Bayes....")
nb_model_1 = test_algorithm("NB")
print("Testing SVM....")
print("Temporarily Skipping SVM....")
#test_algorithm("SVM")

Testing Logistic Regression....
Training the logistic regression model...
Completed!
93.23587869462075
Testing Naive Bayes....
Training Naive Bayes model...
Completed!
93.21664422645381
Testing SVM....
Temporarily Skipping SVM....


In [16]:
# Dump predictive models for dataset 1
joblib.dump(lg_model_1, "./lg_model_1.joblib", compress=True)
joblib.dump(nb_model_1, "./nb_model_1.joblib", compress=True)
joblib.dump(vectorizer1, "./vectorizer_1.joblib", compress=True)
joblib.dump(tfidf1, "./tfidf_1.joblib", compress=True)

['./tfidf_1.joblib']

In [17]:
# Second Driver Program 

#Load New Data
print("Loading Amazon DataSet... ")
df = pd.read_csv('Reviews.csv')
df['sentiment_label'] = df.apply (lambda row: label_sentiment(row, "Score"), axis=1)
df = df.filter(['Text', 'sentiment_label'])

print("Cleaning DataSet...")
#Clean the text column
ct = CleanText()
sr_clean = ct.fit_transform(df['Text'])

# Create clean_text column
df_model = df
df_model['clean_text'] = sr_clean
df_model.columns.tolist()

print("Splitting Dataset....")
# Split the data(cleaned, preprocessed)
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment_label', axis=1), df_model.sentiment_label, test_size=0.20, random_state=2)

vectorizer, vocab, train_data_features, tfidf_features, tfidf  = (
        create_bag_of_words(X_train['clean_text']))

vectorizer2 = vectorizer
tfidf2 = tfidf

print("Testing Logistic Regression....")
lg_model_2 = test_algorithm("LG")
print("Testing Naive Bayes....")
print("Temporarily Skippung NB....")
#test_algorithm("NB")
print("Testing SVM....")
print("Temporarily Skipping SVM....")

Loading Amazon DataSet... 
Cleaning DataSet...
Splitting Dataset....
Creating bag of words...
Completed bag of words...
 Returning vectorizer, vocab, train_data_features, tfidf_features, tfidf.....
Testing Logistic Regression....
Training the logistic regression model...


/Users/ds070111/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Completed!
87.4589897177437
Testing Naive Bayes....
Temporarily Skippung NB....
Testing SVM....
Temporarily Skipping SVM....


In [19]:
# Dump the predictive models for data set 2
joblib.dump(lg_model_2, "./lg_model2.joblib", compress=True)
joblib.dump(vectorizer2, "./vectorizer2.joblib", compress=True)
joblib.dump(tfidf2, "./tfidf2.joblib", compress=True)

['./tfidf2.joblib']

In [31]:
# data = ["""Yuk and yuk! It tastes like something from dollar store. I like a cup of coffe, just a plain cup of coffee and this is brown water. For all you Massachusetts people, this is nothing like the dunkin coffee you get. This is packaged dunkin but it should be generic coffee. I’m sure Folgers is better"""]

# user_data_df = pd.DataFrame(data, columns = ['text']) 
        
# user_data_features = vectorizer2.transform(user_data_df['text'])

# user_data_tfidf_features = tfidf2.fit_transform(user_data_features)

# lg_model_2.predict(user_data_tfidf_features)[0]
